In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!cp /content/drive/MyDrive/bitirme/dataset/phoenix-2014-T.v3.tar.gz /content/phoenix-2014-T.v3.tar.gz

In [3]:
!tar -xzvf /content/phoenix-2014-T.v3.tar.gz -C /content/

Görüntülenen çıkış son 5000 satıra kısaltıldı.
PHOENIX-2014-T-release-v3/PHOENIX-2014-T/features/fullFrame-210x260px/dev/29September_2011_Thursday_heute-4235/images0002.png
PHOENIX-2014-T-release-v3/PHOENIX-2014-T/features/fullFrame-210x260px/dev/29September_2011_Thursday_heute-4235/images0042.png
PHOENIX-2014-T-release-v3/PHOENIX-2014-T/features/fullFrame-210x260px/dev/29September_2011_Thursday_heute-4235/images0039.png
PHOENIX-2014-T-release-v3/PHOENIX-2014-T/features/fullFrame-210x260px/dev/29September_2011_Thursday_heute-4235/images0045.png
PHOENIX-2014-T-release-v3/PHOENIX-2014-T/features/fullFrame-210x260px/dev/29September_2011_Thursday_heute-4235/images0037.png
PHOENIX-2014-T-release-v3/PHOENIX-2014-T/features/fullFrame-210x260px/dev/29September_2011_Thursday_heute-4235/images0030.png
PHOENIX-2014-T-release-v3/PHOENIX-2014-T/features/fullFrame-210x260px/dev/29September_2011_Thursday_heute-4235/images0021.png
PHOENIX-2014-T-release-v3/PHOENIX-2014-T/features/fullFrame-210x260px/d

In [4]:
import os
import shutil
import h5py
import numpy as np
from PIL import Image
from tqdm import tqdm
from concurrent.futures import ProcessPoolExecutor

# 3) Helper to load & transpose a single image
def load_and_transpose(path):
    img = Image.open(path).convert('RGB')
    arr = np.array(img)           # H x W x 3
    return arr.transpose(2, 0, 1) # 3 x H x W

# 4) Main builder function
def build_hdf5(
    data_root: str,
    local_path: str,
    splits=['train', 'dev', 'test'],
    batch_size: int = 64,
    num_workers: int = 4
):
    """
    Walk data_root/{split}/*/*.png/.jpg, build a single .h5 with:
      - one‐image chunks
      - LZF compression
      - batched writes
      - optional parallel decoding
    """
    with h5py.File(local_path, 'w') as h5f:
        for split in splits:
            split_dir = os.path.join(data_root, split)
            img_paths = []
            for sub in sorted(os.listdir(split_dir)):
                subdir = os.path.join(split_dir, sub)
                if not os.path.isdir(subdir):
                    continue
                for fname in sorted(os.listdir(subdir)):
                    if fname.lower().endswith(('.png', '.jpg', '.jpeg')):
                        img_paths.append(os.path.join(subdir, fname))

            n = len(img_paths)
            if n == 0:
                print(f"[{split}] No images found, skipping.")
                continue

            # infer H, W from the first image
            img0 = Image.open(img_paths[0]).convert('RGB')
            H, W = img0.height, img0.width

            # create dataset: one‐image chunk, LZF compression
            dset = h5f.create_dataset(
                split,
                shape=(n, 3, H, W),
                dtype='uint8',
                chunks=(1, 3, H, W),
                compression=None
            )

            # process in batches
            for i in tqdm(range(0, n, batch_size), desc=f"Writing {split}"):
                batch_paths = img_paths[i:i+batch_size]

                # parallel decode?
                if num_workers > 1:
                    with ProcessPoolExecutor(max_workers=num_workers) as exe:
                        arrs = list(exe.map(load_and_transpose, batch_paths))
                    batch_arr = np.stack(arrs, axis=0)  # (B,3,H,W)
                else:
                    batch_arr = np.stack(
                        [load_and_transpose(p) for p in batch_paths],
                        axis=0
                    )

                dset[i : i + len(batch_arr), ...] = batch_arr

            print(f"[{split}] Wrote {n} images ({H}×{W})")

In [6]:
data_root = '/content/PHOENIX-2014-T-release-v3/PHOENIX-2014-T/features/fullFrame-210x260px'
out_h5    = '/content/hdf5-raw'
build_hdf5(data_root, out_h5)
print(f"HDF5 dataset written to {out_h5}")

Writing train: 100%|██████████| 12928/12928 [2:17:28<00:00,  1.57it/s]


[train] Wrote 827354 images (260×210)


Writing dev:  73%|███████▎  | 634/872 [13:05<04:54,  1.24s/it]


OSError: [Errno 12] Cannot allocate memory

In [ ]:
!cp /content/hdf5-raw /content/drive/MyDrive/bitirme/dataset/hdf5-raw-final